<a href="https://colab.research.google.com/github/anelsamadulla/energy_consumption/blob/main/%D0%AD%D0%BD%D0%B5%D1%80%D0%B3%D0%BE%D0%BF%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# В данном файле взята телеметрия за последние 30 дней, без агрегации и с максимальным значением 50000.

In [22]:
import pandas as pd

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
files = {
    "Казан": "sample_data/Казан.csv",
    "Печь": "sample_data/Печь.csv",
    "Плита": "sample_data/Плита.csv",
    "Счетчик_4": "sample_data/Счетчик_4.csv",
    "Счетчик_7": "sample_data/Счетчик_7.csv",
    "Счетчик_17": "sample_data/Счетчик_17.csv",
}

def to_num(s):
    return pd.to_numeric(
        s.astype(str)
         .str.replace(" ", "", regex=False)
         .str.replace(",", ".", regex=False)
         .str.replace("—", "", regex=False)
         .str.replace("-", "", regex=False),
        errors="coerce"
    )

cleaned_list = []

for device, path in files.items():
    df = pd.read_csv(path, sep=";")
    df.columns = df.columns.str.strip()

    # Timestamp
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

    # total_power -> число
    if "total_power" in df.columns:
        df["total_power"] = to_num(df["total_power"])
    else:
        df["total_power"] = pd.NA

    # если total_power пустой — пересчитать из фаз (если есть)
    phases = [c for c in ["p_l1", "p_l2", "p_l3"] if c in df.columns]
    if df["total_power"].isna().all() and len(phases) > 0:
        for c in phases:
            df[c] = to_num(df[c])
        df["total_power"] = df[phases].sum(axis=1, min_count=1)

    # оставить только нужное
    df = df[["Timestamp", "total_power"]].copy()
    df["device"] = device

    # убрать мусорные строки
    df = df.dropna(subset=["Timestamp", "total_power"])
    df = df.sort_values("Timestamp").reset_index(drop=True)

    cleaned_list.append(df)

all_devices = pd.concat(cleaned_list, ignore_index=True)

all_devices.head()

,Timestamp,total_power,device
0,2026-02-18 21:34:42,4.85,Казан
1,2026-02-18 21:34:49,4.22,Казан
2,2026-02-18 21:34:57,4.36,Казан
3,2026-02-18 21:35:05,3.55,Казан
4,2026-02-18 21:35:12,3.59,Казан


In [25]:
len(all_devices)

300000

In [26]:
all_devices.shape[0]

300000

In [27]:
all_devices.isnull().sum()

,0
Timestamp,0
total_power,0
device,0


In [28]:
all_devices.groupby("device").size()

,0
device,
Казан,50000
Печь,50000
Плита,50000
Счетчик_17,50000
Счетчик_4,50000
Счетчик_7,50000


In [11]:
all_devices.groupby("device")["total_power"].agg(["count", "min", "max", "mean"])

,count,min,max,mean
device,,,,
Казан,50000,0.00,6528.11,884.280965
Печь,50000,0.00,11590.58,925.304172
Плита,50000,0.00,14103.63,10159.820696
Счетчик_17,50000,193.17,217.78,200.118720
Счетчик_4,50000,112.86,2634.98,951.329811
Счетчик_7,50000,165.34,1435.15,303.486072


In [12]:
def calculate_losses(df, standby_power, min_duration_min=10, tariff_kzt_per_kwh=120):

    df = df.sort_values("Timestamp").copy()

    # статус включения
    df["is_on"] = df["total_power"] > standby_power

    # блоки включения/выключения
    df["block"] = (df["is_on"] != df["is_on"].shift()).cumsum()

    events = []

    for (device, block), group in df.groupby(["device", "block"]):

        if not group["is_on"].iloc[0]:
            continue

        start = group["Timestamp"].iloc[0]
        end = group["Timestamp"].iloc[-1]

        duration_min = (end - start).total_seconds() / 60

        if duration_min >= min_duration_min:
            avg_power_w = group["total_power"].mean()
            hours = duration_min / 60
            loss_kzt = (avg_power_w / 1000) * hours * tariff_kzt_per_kwh

            events.append({
                "device": device,
                "start": start,
                "end": end,
                "duration_min": round(duration_min, 2),
                "avg_power_w": round(avg_power_w, 2),
                "loss_kzt": round(loss_kzt, 2)
            })

    return pd.DataFrame(events)

In [13]:
losses = calculate_losses(
    all_devices,
    standby_power=100,      # ниже 100W считаем standby
    min_duration_min=15,    # минимум 15 минут
    tariff_kzt_per_kwh=120
)

losses.head()

,device,start,end,duration_min,avg_power_w,loss_kzt
0,Казан,2026-02-19 13:29:58,2026-02-19 13:51:26,21.47,6438.80,276.44
1,Казан,2026-02-20 10:16:50,2026-02-20 10:43:28,26.63,3593.96,191.44
2,Казан,2026-02-21 09:55:53,2026-02-21 10:26:40,30.78,6087.11,374.76
3,Печь,2026-02-19 13:30:00,2026-02-19 13:51:20,21.33,3824.21,163.17
4,Печь,2026-02-20 10:16:52,2026-02-20 10:43:30,26.63,4318.36,230.02


In [14]:
losses.groupby("device")["loss_kzt"].sum()

,loss_kzt
device,
Казан,842.64
Печь,680.90
Плита,85608.53
Счетчик_17,31.50
Счетчик_4,111.35
Счетчик_7,30.02


In [15]:
losses["loss_kzt"].sum()

np.float64(87304.94)

In [ ]:
#Второе вычисление

In [16]:
START = "18:00:00"
END = "06:00:00"
TARIFF = 40  # ₸ за кВт⋅ч

df = all_devices.copy()
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df = df.sort_values(["device", "Timestamp"])

# выделяем время
time = df["Timestamp"].dt.time

is_after_18 = time >= pd.to_datetime(START).time()
is_before_6 = time < pd.to_datetime(END).time()

# работа вне рабочего времени
df["is_violation"] = (is_after_18 | is_before_6) & (df["total_power"] > 0)

In [17]:
df["dt_hours"] = df.groupby("device")["Timestamp"].diff().dt.total_seconds().div(3600)
df["dt_hours"] = df["dt_hours"].fillna(0)

df["kwh"] = (df["total_power"] / 1000) * df["dt_hours"]

violations = df[df["is_violation"]].copy()
violations["loss_kzt"] = violations["kwh"] * TARIFF

In [18]:
violations.groupby("device")[["kwh", "loss_kzt"]].sum()

,kwh,loss_kzt
device,,
Казан,2.640759,105.630357
Печь,5.997927,239.917074
Плита,23.608573,944.342937
Счетчик_17,11.061507,442.460266
Счетчик_4,51.725961,2069.038442
Счетчик_7,16.486205,659.448198


In [19]:
violations["loss_kzt"].sum()

np.float64(4460.83727588889)

In [20]:
all_devices.groupby("device")["total_power"].max()

,total_power
device,
Казан,6528.11
Печь,11590.58
Плита,14103.63
Счетчик_17,217.78
Счетчик_4,2634.98
Счетчик_7,1435.15


In [21]:
violations.groupby("device")["dt_hours"].sum() * 60

,dt_hours
device,
Казан,3384.616667
Печь,3592.716667
Плита,593.700000
Счетчик_17,3309.950000
Счетчик_4,3239.383333
Счетчик_7,3237.783333
